In [ ]:
SYFT_VERSION = ">=0.8.2.b0,<0.9"
package_string = f'"syft{SYFT_VERSION}"'
# %pip install {package_string} -q

In [ ]:
# stdlib
import io
import os
import time

# third party
import pandas as pd

# syft absolute
import syft as sy
from syft import autocache

sy.requires(SYFT_VERSION)

In [ ]:
node = sy.orchestra.launch(name="test-domain-1", port="auto", dev_mode=True)

In [ ]:
domain_client = node.login(email="info@openmined.org", password="changethis")

## From file using Action Object (Partial Functional)

In [ ]:
# creating an action object from file
action_object = sy.ActionObject.from_path(path=data_file)

In [ ]:
data_ptr = action_object.send(domain_client)

In [ ]:
with open(action_object.syft_action_data.file_name, "wb") as f:
    f.write(action_object.syft_action_data.read())

In [ ]:
max_wait = 5
start = time.time()

In [ ]:
while not os.path.exists(data_ptr.syft_action_data.file_name):
    print("waiting...")
    time.sleep(1)
    if start + max_wait <= time.time():
        print("timeout reached")
        break

In [ ]:
@sy.syft_function_single_use(trade_data=data_ptr)
def sum_trade_value_mil(trade_data):
    # third party
    import opendp.prelude as dp
    import pandas as pd

    dp.enable_features("contrib")

    aggregate = 0.0
    base_lap = dp.m.make_base_laplace(
        dp.atom_domain(T=float),
        dp.absolute_distance(T=float),
        scale=10.0,
    )

    noise = base_lap(aggregate)
    print("Loading file:", data_ptr.syft_action_data.file_name)
    df = pd.read_csv(data_ptr.syft_action_data.file_name, low_memory=False)
    total = df["Trade Value (US$)"].sum()
    return (float(total / 1_000_000), float(noise))

In [ ]:
if os.path.exists(data_ptr.syft_action_data.file_name):
    sum_trade_value_mil(trade_data=data_ptr)
else:
    raise Exception(f"File {data_file} doesn't exist when calling sum_trade_value_mil.")

In [ ]:
## clean up
## delete downloaded file

## delete file from blob storage
domain_client.api.services.blob_storage.delete(action_object.syft_blob_storage_entry_id)

time.sleep(1)

if os.path.exists(data_ptr.syft_action_data.file_name):
    print(f"Trying to delete file: {data_ptr.syft_action_data.file_name}")
    os.unlink(data_ptr.syft_action_data.file_name)

if os.path.exists(retrieved_file):
    print(f"Trying to delete file: {retrieved_file}")
    os.unlink(retrieved_file)

In [ ]:
# Cleanup local domain server
if node.node_type.value == "python":
    node.land()